# 第四章 模型链

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、大语言模型链](#二、大语言模型链)
     - [2.1 导入数据](#2.1-导入数据)
     - [2.2 构建大语言模型链](#2.2-构建大语言模型链)
 - [三、顺序链](#三、顺序链)
     - [3.1 简单顺序链](#3.1-简单顺序链)
     - [3.2 顺序链](#3.2-顺序链)
         - [3.2.1 创建四个子链](#3.2.1-创建四个子链)
         - [3.2.2 对四个子链进行组合](#3.2.2-对四个子链进行组合)
 - [四、 路由链](#四、-路由链)
     - [4.1 定义提示模板](#4.1-定义提示模板)
     - [4.2 对提示模版进行命名和描述](#4.2-对提示模版进行命名和描述)
     - [4.3 基于提示模版信息创建相应目标链  ](#4.3-基于提示模版信息创建相应目标链--)
     - [4.4 创建默认目标链](#4.4-创建默认目标链)
     - [4.5 定义不同链之间的路由模板](#4.5-定义不同链之间的路由模板)
     - [4.6 构建路由链](#4.6-构建路由链)
     - [4.7 创建整体链路](#4.7-创建整体链路)
     - [4.8 进行提问](#4.8-进行提问)
     - [4.9 中文提示](#4.9-中文提示)


链（Chains）通常将大语言模型（LLM）与提示(Prompt)结合在一起，基于此，我们可以对文本或数据进行一系列操作。

链（Chains）可以一次性接受多个输入

例如，我们可以创建一个链，该链接受用户输入，使用提示模板对其进行格式化，然后将格式化的响应传递给LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起来构建更复杂的链。


Python在格式化字符串时，双括号会被解析为单个括号。


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

## 二、大语言模型链

大语言模型链(LLMChain)是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

### 2.1 导入数据

In [1]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [2]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### 2.2 构建大语言模型链

In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate  
from langchain.chains.llm import LLMChain


# 初始化语言模型
API_KEY = os.environ.get("CHAT_ANYWHERE_API_KEY")
BASE_URL = "https://api.chatanywhere.com.cn/v1"
llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)

# 初始化提示模版
prompt = ChatPromptTemplate.from_template("根据产品名称```{product}```为制作该产品的公司取一个合适的公司名称?")

# 将大语言模型(LLM)和提示（Prompt）组合成链
chain = LLMChain(llm=llm, prompt=prompt)
product = "加大双人床床单套装"
chain.run(product)

'舒适家居用品制造有限公司'

## 三、顺序链

### 3.1 简单顺序链

顺序链（SequentialChains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。

In [10]:
# from langchain.chains import SimpleSequentialChain
from langchain.chains.sequential import SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain


# 1.初始化大语言模型
llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)

# 2.创建两个子链
# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(
    "根据产品名称```{product}```为制作该产品的公司取一个合适的公司名称?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(
    "对于下面这个公司：```{company_name}```，写一个20字的描述。"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# 3.构建简单顺序链
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)
product = "大号床单套装"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
豪华纺织品有限公司
豪华纺织品有限公司提供高品质纺织产品，致力于为客户打造舒适、时尚的家居生活。

> Finished chain.


'豪华纺织品有限公司提供高品质纺织产品，致力于为客户打造舒适、时尚的家居生活。'

### 3.2 顺序链

当只有一个输入和一个输出时，简单顺序链（SimpleSequentialChain）即可实现。当有多个输入或多个输出时，我们则需要使用顺序链（SequentialChain）来实现。

让我们选择一篇评论并通过整个链传递它，可以发现，原始review是法语，可以把英文review看做是一种翻译，接下来是根据英文review得到的总结，最后输出的是用法语原文进行的续写信息。

In [11]:
# from langchain.chains import SequentialChain
from langchain.chains.sequential import SequentialChain
from langchain_openai import ChatOpenAI   #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains.llm import LLMChain    #导入LLM链。


# 1.初始化大语言模型
llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)

#### 3.2.1 创建四个子链

In [12]:
#输入：review
#输出：英文review，总结，后续回复

#子链1
# prompt模板 1: 翻译成英语（把下面的review翻译成中文）
first_prompt = ChatPromptTemplate.from_template(
    "把三个反引号中的文本翻译成中文:\n```{Review}```"
)
# chain 1: 输入：Review    输出：中文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="Chinese_Review")

#子链2
# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "请你用一句话来总结三个反引号中的文本:\n```{Chinese_Review}```"
)
# chain 2: 输入：中文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


#子链3
# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "三个反引号中的文本使用的是什么语言:\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


#子链4
# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "使用特定的语言对下面的总结写一个后续回复:"
    "\n\n总结: {summary}\n\n语言: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message")

#### 3.2.2 对四个子链进行组合

In [15]:
# 对四个子链进行组合
#输入：review    输出：英文review，总结，后续回复
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Chinese_Review", "summary","followup_message"],
    verbose=True
)


review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'Chinese_Review': '我觉得味道很差。泡沫不持久，很奇怪。我在商店买的同样产品味道好多了...\n是老货还是假货！？',
 'summary': '对于这款产品，味道差、泡沫不持久，怀疑是老货或者假货。',
 'followup_message': "Bonjour! Merci pour votre retour sur le produit. Nous sommes désolés d'apprendre que vous n'êtes pas satisfait de la qualité. Nous allons enquêter sur la question pour nous assurer que nos produits sont authentiques et frais. Veuillez nous contacter si vous avez d'autres préoccupations. Merci!"}

## 四、 路由链

到目前为止，我们已经学习了大语言模型链和顺序链。但是，如果我们想做一些更复杂的事情怎么办？

一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

- 路由链（Router Chain）：路由器链本身，负责选择要调用的下一个链
- destination_chains：路由器链可以路由到的链

举一个具体的例子，让我们看一下我们在不同类型的链之间路由的地方，我们在这里有不同的prompt:  

In [16]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate


llm = ChatOpenAI(temperature=0.0, base_url=BASE_URL, api_key=API_KEY)

### 4.1 定义提示模板

首先，我们定义提示适用于不同场景下的提示模板。

In [17]:
#第一个提示适合回答物理问题
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认你不知道。

这是一个问题:
{input}"""


#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""


#第三个适合回答历史问题
history_template = """你是一个非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这还是一个输入：
{input}"""

### 4.2 对提示模版进行命名和描述

在定义了这些提示模板后，我们可以为每个模板命名，并给出具体描述。例如，第一个物理学的描述适合回答关于物理学的问题，这些信息将传递给路由链，然后由路由链决定何时使用此子链。

In [19]:
prompt_infos = [
    {
        "名字": "物理学",
        "描述": "擅长回答关于物理学的问题",
        "提示模板": physics_template
    },
    {
        "名字": "数学",
        "描述": "擅长回答数学问题",
        "提示模板": math_template
    },
    {
        "名字": "历史",
        "描述": "擅长回答历史问题",
        "提示模板": history_template
    },
    {
        "名字": "计算机科学",
        "描述": "擅长回答计算机科学问题",
        "提示模板": computerscience_template
    }
]

LLMRouterChain（此链使用 LLM 来确定如何路由事物）

在这里，我们需要一个**多提示链**。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。 但是这只是路由的一种类型，我们也可以在任何类型的链之间进行路由。

这里我们要实现的几个类是大模型路由器链。这个类本身使用语言模型来在不同的子链之间进行路由。这就是上面提供的描述和名称将被使用的地方。

### 4.3 基于提示模版信息创建相应目标链  
目标链是由路由链调用的链，每个目标链都是一个语言模型链

In [32]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["名字"]
    prompt_template = p_info["提示模板"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['名字']}: {p['描述']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

### 4.4 创建默认目标链
除了目标链之外，我们还需要一个默认目标链。这是一个当路由器无法决定使用哪个子链时调用的链。在上面的示例中，当输入问题与物理、数学、历史或计算机科学无关时，可能会调用它。

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

### 4.5 定义不同链之间的路由模板

这包括要完成的任务的说明以及输出应该采用的特定格式。

注意：此处在原教程的基础上添加了一个示例，主要是因为"gpt-3.5-turbo"模型不能很好适应理解模板的意思，使用 "text-davinci-003" 或者"gpt-4-0613"可以很好的工作，因此在这里多加了示例提示让其更好的学习。
eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

In [48]:
# 多提示路由模板
# 在格式化字符串时，双括号会被解析为单个括号。
MULTI_PROMPT_ROUTER_TEMPLATE = """给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```


记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太适合任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```

"""

### 4.6 构建路由链
首先，我们通过格式化上面定义的目标创建完整的路由器模板。这个模板可以适用许多不同类型的目标。
因此，在这里，您可以添加一个不同的学科，如英语或拉丁语，而不仅仅是物理、数学、历史和计算机科学。

接下来，我们从这个模板创建提示模板

最后，通过传入llm和整个路由提示来创建路由链。需要注意的是这里有路由输出解析，这很重要，因为它将帮助这个链路决定在哪些子链路之间进行路由。

In [49]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)
router_prompt.template

'给语言模型一个原始文本输入，让其选择最适合输入的模型提示。系统将为您提供可用提示的名称以及最适合改提示的描述。如果你认为修改原始输入最终会导致语言模型做出更好的响应，你也可以修改原始输入。\n\n\n<< 格式 >>\n返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：\n```json\n{{\n    "destination": 字符串 \\ 使用的提示名字或者使用 "DEFAULT"\n    "next_inputs": 字符串 \\ 原始输入的改进版本\n}}\n```\n\n\n记住：“destination”必须是下面指定的候选提示名称之一，或者如果输入不太适合任何候选提示，则可以是 “DEFAULT” 。\n记住：如果您认为不需要任何修改，则 “next_inputs” 可以只是原始输入。\n\n<< 候选提示 >>\n物理学: 擅长回答关于物理学的问题\n数学: 擅长回答数学问题\n历史: 擅长回答历史问题\n计算机科学: 擅长回答计算机科学问题\n\n<< 输入 >>\n{input}\n\n<< 输出 (记得要包含 ```json)>>\n\n样例:\n<< 输入 >>\n"什么是黑体辐射?"\n<< 输出 >>\n```json\n{{\n    "destination": 字符串 \\ 使用的提示名字或者使用 "DEFAULT"\n    "next_inputs": 字符串 \\ 原始输入的改进版本\n}}\n```\n\n'

### 4.7 创建整体链路

In [50]:
#多提示链
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

### 4.8 进行提问

如果我们问一个物理问题，我们希望看到他被路由到物理链路

In [52]:
# 问题：什么是黑体辐射？
chain.run("什么是黑体辐射？")



> Entering new MultiPromptChain chain...
物理学: {'input': '请解释一下黑体辐射是什么？'}
> Finished chain.


'黑体辐射是指一个理想化的物体，它能够吸收所有入射到它表面的辐射能量，并且以最高效率地辐射出来。根据黑体辐射定律，黑体辐射的辐射强度与波长和温度有关，遵循普朗克辐射定律和斯蒂芬-玻尔兹曼定律。黑体辐射是研究热辐射和量子力学的基础，对于理解物体的热辐射特性和热力学过程具有重要意义。希望这个解释对你有所帮助。'

如果我们问一个数学问题，我们希望看到他被路由到数学链路

In [54]:
# 问题：2+2等于多少？
chain.run("2 + 2 等于多少")



> Entering new MultiPromptChain chain...
数学: {'input': '求解 2 + 2'}
> Finished chain.


'2 + 2 = 4.'

如果我们传递一个与任何子链路都无关的问题时，会发生什么呢？

这里，我们问了一个关于生物学的问题，我们可以看到它选择的链路是无。这意味着它将被**传递到默认链路，它本身只是对语言模型的通用调用**。语言模型幸运地对生物学知道很多，所以它可以帮助我们。

In [55]:
# 问题：为什么我们身体里的每个细胞都包含DNA？
chain.run("为什么我们身体里的每个细胞都包含DNA?")



> Entering new MultiPromptChain chain...
物理学: {'input': '为什么我们身体里的每个细胞都包含DNA?'}
> Finished chain.


'每个细胞都包含DNA是因为DNA携带着遗传信息，控制着细胞的生长、分裂和功能。DNA是细胞内的遗传物质，它包含了编码蛋白质的基因序列。这些基因序列决定了细胞的特性和功能。因此，每个细胞都需要包含DNA来确保细胞的正常运作和遗传信息的传递。'

In [56]:
# 中文
chain.run("你知道李白是谁嘛?")



> Entering new MultiPromptChain chain...
历史: {'input': '你知道李白是谁吗?'}
> Finished chain.


'是的，李白是中国唐代著名的诗人，被誉为“诗仙”。他的诗作以豪放、奔放、豪情和浪漫著称，被后人称为“诗圣”。李白的诗歌广泛流传，对后世文学产生了深远的影响。他的作品中常常表现出对自然、人生、酒和友谊的热爱，被视为中国古代文学的瑰宝之一。'